# TRY 1

## 将实例化的逻辑表达式转化为自然语言

In [ ]:
from typing import List
from typing import Optional
from typing import Dict

import os
import json
from pathlib import Path

import httpx
from openai import OpenAI

import requests

import time
import re
from tqdm import tqdm

## 逻辑表达式转化为自然语言

In [ ]:
# 文件路径
file_path = "prompts/DataFountain/nl.txt"

# 请求的 URL
url = "https://api.moonshot.cn/v1/files"

headers = {
    'Authorization': 'sk-gNToXJPO4Ih8NDZnDloSXR780dQDEZ2Epu49wcVA1sWb4Ecw'
}

# 准备文件数据
files = {'file': open(file_path, 'rb')}

# 发起 POST 请求
response = requests.post(url, headers=headers, files=files)

# 打印响应内容
print(response.status_code)
print(response.text)

In [ ]:
client = OpenAI(
    api_key="sk-gNToXJPO4Ih8NDZnDloSXR780dQDEZ2Epu49wcVA1sWb4Ecw",
    base_url="https://api.moonshot.cn/v1",
)

file_object = client.files.create(file=Path("prompts/DataFountain/nl.txt"), purpose="file-extract")
file_content = client.files.content(file_id=file_object.id).text

In [ ]:
with open('data/DF/instance_predicates_v3.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

In [ ]:
print(len(data))

In [ ]:
def extract_json(text):
    # 使用正则表达式匹配并提取 JSON 数据（假设 JSON 数据用 `{` 开头和 `}` 结尾）
    json_data = re.search(r'\[.*\]', text, re.DOTALL)

    # 检查是否找到匹配的 JSON 数据
    if json_data:
        json_string = json_data.group(0)

        # 将 JSON 数据解析为字典
        data = json.loads(json_string)
        return data
    else:
        print("未找到 JSON 数据。")

In [ ]:
print(extract_json(data[0]))

## 使用api转化自然语言

In [ ]:
def call_api_with_rate_limit_handling(client, messages, model="moonshot-v1-32k", max_retries=5):
    retries = 0
    while retries < max_retries:
        try:
            completion = client.chat.completions.create(
                model=model,
                messages=messages,
                temperature=0,
            )
            return completion
        except Exception as e:
            if '429' in str(e):  # 检查异常消息中是否包含429
                print("Rate limit reached. Retrying in 1 second...")
                time.sleep(1)  # 等待1秒钟
                retries += 1
            else:
                raise  # Re-raise the exception if it's not a 429 error
    raise Exception("Max retries reached. API call failed.")

In [ ]:
nl = []

In [ ]:
for dicts in tqdm(data, desc='rule转化逻辑表达式进度'):
    # 替换内容
    # 使用正则表达式替换 [[RULES]]
    # content = re.sub(r'\[\[RULES\]\]', instance, file_content)
    # print(content)

    # 使用replace
    content = file_content.replace("[[RULES]]", dicts)

    messages = [
        {
            "role": "system",
            "content": "你是 Kimi，由 Moonshot AI 提供的人工智能助手，你更擅长中文和英文的对话。你会为用户提供安全，有帮助，准确的回答。同时，你会拒绝一切涉及恐怖主义，种族歧视，黄色暴力等问题的回答。Moonshot AI 为专有名词，不可翻译成其他语言。请将内容输出为可解析的json格式。"
        },
        {
            "role": "system",
            "content": content,
        },
        {
            "role": "user",
            "content": "你好，请根据txt内容生成回答。"
        }
    ]

    completion = call_api_with_rate_limit_handling(client, messages, "moonshot-v1-32k", 5)

    nl.append(completion.choices[0].message.content)

In [ ]:
print(nl[0])

In [ ]:
with open('data/DF/instance_nl_v1.json', 'w', encoding='utf-8') as f:
    json.dump(nl, f, ensure_ascii=False, indent=4)

# 使用缓存

In [1]:
from typing import List
from typing import Optional
from typing import Dict

import os
import json
import jsonlines
from pathlib import Path

import httpx
from openai import OpenAI

import requests

import time
import re
from tqdm import tqdm

In [2]:
# 你的API密钥
api_key = "sk-gNToXJPO4Ih8NDZnDloSXR780dQDEZ2Epu49wcVA1sWb4Ecw"

# 读取文本文件内容
with open('prompts/DataFountain/nl.txt', 'r', encoding='utf-8') as file:
    file_content = file.read()

# 创建缓存的请求数据
data = {
    "model": "moonshot-v1",
    "messages": [
        {
            "role": "system",
            "content": file_content
        }
    ],
    "name": "example_cache",
    "ttl": 3600  # 缓存有效期，单位为秒
}

# 发送POST请求创建缓存
response = requests.post(
    url="https://api.moonshot.cn/v1/caching",
    headers={
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    },
    json=data
)

# 打印响应内容
print(json.loads(response.text))

{'id': 'cache-ezijs7moc6di11gdge8i', 'object': 'cache_object', 'model': 'moonshot-v1', 'messages': [{'role': 'system', 'content': '任务描述：你将被给予一个列表，列表每个元素是一条 JSON 格式的数据，包含：1. 谓词的描述和谓词的具体取值（instance_predicates）；2. 将这些具体取值的谓词代入逻辑表达式得到的表达式（instance_logicexpression）。你的任务是将实例化的逻辑表达式，转化为自然语言的形式。\n------\n下面是一个例子：\n```json\n[\n    {\n        "instance_predicates": [\n            "s=“化工厂” — 表示事故场景在化工厂",\n            "i=“张三” — 表示安全监察专员为张三",\n            "r=“化工事故应急处理规定” — 表示领导发出的指示和相关规定"\n        ],\n        "instance_logicexpression": [\n            "∀s∀i∀r\xa0(Accident(“化工厂”)∧Instruction(Leader,\xa0“化工事故应急处理规定”)→(GoTo(“张三”,\xa0“化工厂”)∧Participate(“张三”,\xa0“化工厂”,\xa0“应急救援”)∧Participate(“张三”,\xa0“化工厂”,\xa0“调查”)))"\n        ]\n    },\n    {\n        "instance_predicates": [\n            "s=“矿井” — 表示事故场景在矿井",\n            "i=“李四” — 表示安全监察专员为李四",\n            "r=“矿山安全生产规章” — 表示领导发出的指示和相关规定"\n        ],\n        "instance_logicexpression": [\n            "∀s∀i∀r\xa0(Accident(“矿井”)∧Instruction(Leader,\x

In [3]:
# 读取文本文件内容
def load_file_content(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()


# 创建缓存
def create_cache(file_content):
    data = {
        "model": "moonshot-v1",
        "messages": [
            {
                "role": "system",
                "content": file_content
            }
        ],
        "name": "example_cache",
        "ttl": 3600  # 缓存有效期，单位为秒
    }
    response = requests.post(
        url="https://api.moonshot.cn/v1/caching",
        headers={
            "Authorization": f"Bearer {api_key}",
            "Content-Type": "application/json"
        },
        json=data
    )
    cache_response = json.loads(response.text)
    return cache_response['id']


# 检查缓存是否存在且未过期
def check_cache(cache_id):
    response = requests.get(
        url=f"https://api.moonshot.cn/v1/caching/{cache_id}",
        headers={
            "Authorization": f"Bearer {api_key}"
        }
    )
    if response.status_code == 200:
        return True
    return False


# 重新加载数据并更新缓存
def reload_and_update_cache(file_path, cache_id):
    new_file_content = load_file_content(file_path)
    data = {
        "model": "moonshot-v1",
        "messages": [
            {
                "role": "system",
                "content": new_file_content
            }
        ],
        "name": "example_cache",
        "ttl": 3600  # 缓存有效期，单位为秒
    }
    response = requests.put(
        url=f"https://api.moonshot.cn/v1/caching/{cache_id}",
        headers={
            "Authorization": f"Bearer {api_key}",
            "Content-Type": "application/json"
        },
        json=data
    )
    return response.status_code == 200


# 使用缓存内容并添加问题
def use_cache_with_question(cache_id, question):
    data = {
        "model": "moonshot-v1-32k",
        "messages": [
            {
                "role": "cache",
                "content": f"cache_id={cache_id};reset_ttl=3600",
            },
            {
                "role": "user",
                "content": question
            }
        ]
    }
    response = requests.post(
        url="https://api.moonshot.cn/v1/chat/completions",
        headers={
            "Authorization": f"Bearer {api_key}",
            "Content-Type": "application/json"
        },
        json=data
    )
    return response.json()['choices'][0]['message']['content']

In [8]:
# 主流程
file_path = 'prompts/DataFountain/nl.txt'
file_content = load_file_content(file_path)
print(f"File{file_path} success read")
cache_id = create_cache(file_content)
print(f"cache id:{cache_id}")
print("cache cuccess")

# # 模拟缓存过期
# time.sleep(3600)

# 检查缓存是否存在且未过期
if not check_cache(cache_id):
    print("检查不到缓存")
    # 重新加载数据并更新缓存
    if reload_and_update_cache(file_path, cache_id):
        print("缓存更新成功")
    else:
        print("缓存更新失败")

print("缓存已存在")

# with open('data/法律/law_logic2predicates_v4.json', 'r', encoding='utf-8') as file:
#     answer = json.load(file)
with open('data/法律/law_logic2predicates_v5.jsonl', 'r', encoding='utf-8') as file:
    # 逐行读取
    answer = []
    for line in file:
        # 去掉行尾的换行符
        line = line.strip()
        # 解析 JSON 数据
        line = json.loads(line)
        answer.append(line)
        

for idx, value in tqdm(enumerate(answer), total=len(answer), desc="Processing"):
    context = value['predicates']
    # 使用缓存内容并添加问题
    question = f"请回答关于文件内容的问题，其中[[CONTEXT]]代表的数据为{context}"
    response = use_cache_with_question(cache_id, question)
    judgement = {
        "id": value["id"],
        "number": value['number'],
        "judgement": value['judgement'],
        "predicates": value['predicates'],
        "natural_language": response
    }
    # 打开文件以进行写入，如果文件不存在，会创建文件
    with jsonlines.open('data/法律/law_predicates2nl_v2.jsonl', mode='a') as writer:
        writer.write(judgement)


Fileprompts/DataFountain/nl.txt success read
cache id:cache-ezijuidoc6di11gdgfki
cache cuccess
缓存已存在


Processing: 100%|██████████| 2/2 [00:12<00:00,  6.10s/it]
